In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [2]:
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *

2018-08-24 14:00:59.730 INFO in 'summarizer.preprocessing.cleaner'['textcleaner'] at line 37: 'pattern' package not found; tag filters are not available for English


In [3]:
tf.reset_default_graph()
sess = tf.Session()

In [4]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-08-24 14:01:00.97 INFO in 'tensorflow'['tf_logging'] at line 159: Using /tmp/tfhub_modules to cache modules.


In [5]:
print(tf.trainable_variables())
elmo_params = {'layer_coefficients': tf.trainable_variables()[-2], 'scaling': tf.trainable_variables()[-1]}
print(elmo_params)

[<tf.Variable 'module/bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel:0' shape=(1024, 16384) dtype=float32>, <tf.Variable 'module/bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/bias:0' shape=(16384,) dtype=float32>, <tf.Variable 'module/bilm/RNN_0/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel:0' shape=(4096, 512) dtype=float32>, <tf.Variable 'module/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0' shape=(1024, 16384) dtype=float32>, <tf.Variable 'module/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/bias:0' shape=(16384,) dtype=float32>, <tf.Variable 'module/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0' shape=(4096, 512) dtype=float32>, <tf.Variable 'module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel:0' shape=(1024, 16384) dtype=float32>, <tf.Variable 'module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/bias:0' shape=(16384,) dtype=float32>, <tf.Variable 'module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/ke

In [6]:
tokens_input = tf.placeholder(shape=[None, None], dtype=tf.string)
tokens_length = tf.placeholder(shape=[None], dtype=tf.int32)
y = tf.placeholder(shape=[None, None], dtype=tf.float32)

embeddings = elmo(inputs={"tokens": tokens_input,
                          "sequence_len": tokens_length},
                  signature="tokens",
                  as_dict=True)['elmo']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-24 14:01:00.696 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


In [7]:
test_sen = [['Sun', 'is' ,'rising']]

In [8]:
initialize_op = tf.global_variables_initializer()
sess.run([initialize_op])

[None]

In [12]:
embeddings_res = sess.run([embeddings], feed_dict={tokens_input:test_sen, tokens_length: [len(s) for s in test_sen]})[0]

In [13]:
print(embeddings_res.shape)

(1, 3, 1024)


In [24]:
print(embeddings_res)

[[[-0.11844927 -0.7423455  -0.34201357 ...  0.13534099  0.04311828
    0.93761355]
  [-0.6730251  -0.29672506 -0.00278771 ...  0.07860345  0.83121437
    0.3668858 ]
  [-0.13479187 -0.32009083  0.1162141  ...  0.73134965 -0.00319648
    0.13401002]]]


In [20]:
g_2 = tf.Graph()
sess2 = tf.Session(graph=g_2)
with g_2.as_default():
    elmo2 = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)

In [21]:
print(tf.trainable_variables())

[]


In [22]:
with g_2.as_default():
    embeddings2 = elmo2(inputs={"tokens": tokens_input,
                              "sequence_len": tokens_length},
                      signature="tokens",
                      as_dict=True)['elmo']

ValueError: Tensor("Placeholder_1:0", shape=(?,), dtype=int32) must be from the same graph as Tensor("module/bilm/char_embed:0", shape=(), dtype=resource).

In [23]:
initialize_op = tf.global_variables_initializer()
sess2.run([initialize_op])
embeddings_res2 = sess2.run([embeddings], feed_dict={tokens_input:test_sen, tokens_length: [len(s) for s in test_sen]})[0]

ValueError: Fetch argument <tf.Operation 'init' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "init"
op: "NoOp"
input: "^AssignVariableOp_38"
input: "^AssignVariableOp_2"
input: "^AssignVariableOp_9"
input: "^AssignVariableOp_3"
input: "^AssignVariableOp_10"
input: "^AssignVariableOp_4"
input: "^AssignVariableOp_11"
input: "^AssignVariableOp_5"
input: "^AssignVariableOp_12"
input: "^AssignVariableOp_6"
input: "^AssignVariableOp_13"
input: "^AssignVariableOp_7"
input: "^AssignVariableOp_14"
input: "^AssignVariableOp_8"
input: "^AssignVariableOp_15"
input: "^AssignVariableOp_16"
input: "^AssignVariableOp_18"
input: "^AssignVariableOp_17"
input: "^AssignVariableOp_19"
input: "^AssignVariableOp_20"
input: "^AssignVariableOp_22"
input: "^AssignVariableOp_21"
input: "^AssignVariableOp_23"
input: "^AssignVariableOp_24"
input: "^AssignVariableOp_25"
input: "^AssignVariableOp_27"
input: "^AssignVariableOp_26"
input: "^AssignVariableOp_28"
input: "^AssignVariableOp_30"
input: "^

In [ ]:
print(embeddings_res2.shape)

In [ ]:
print(np.linalg.norm(embeddings_res - embeddings_res2))